<a href="https://colab.research.google.com/github/ffreemt/app1/blob/master/0_ycco_self_llm_04_Baichuan2_7B_chat_Lora_%E5%BE%AE%E8%B0%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

modify Copy of ycco-self-llm-04-Baichuan2-7B-chat Lora 微调.ipynb tp use SFTTrainer
  - ref
    * ycco https://colab.research.google.com/drive/1wlNyPKAKknldZ_06D52LaR38sRLBJ8xk#scrollTo=TSsPNqspeFt5 **Copy of deci-SFTTrainer-LLAMA-2-Fine-Tuning-Lora.ipynb**
    * https://deci.ai/blog/fine-tune-llama-2-with-lora-for-question-answering/ **How to Fine-tune Llama 2 with LoRA for Question Answering: A Guide for Practitioners**

https://github.com/datawhalechina/self-llm/blob/master/BaiChuan/04-Baichuan2-7B-chat%20Lora%20%E5%BE%AE%E8%B0%83.ipynb

https://github.com/jeinlee1991/chinese-llm-benchmark?tab=readme-ov-file#1%E7%BB%BC%E5%90%88%E8%83%BD%E5%8A%9B%E6%8E%92%E8%A1%8C%E6%A6%9C CLiB中文大模型能力评测榜单（持续更新）



imported from ycco  https://colab.research.google.com/drive/1yuKvcUzY4xbefWVWysTCcqSHsDGCxqkz#scrollTo=f142cb9c-ad99-48e6-ba86-6df198f9ed96

# prelim

In [ ]:
# @title prelim

try:
  import torch
except ModuleNotFoundError:
  !pip install torch -qq
  import torch
assert torch.cuda.is_available(), "no GPU. turon on GPU and try again. 没GPU，基本玩不了。"

%env PIP_ROOT_USER_ACTION ignore
%env PYTHONWARNINGS ignore
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from IPython.display import clear_output as clear, HTML

%pip install -Uqq pip

env: PIP_ROOT_USER_ACTION=ignore
env: PYTHONWARNINGS=ignore
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.4 MB/s eta 0:00:00


# platform, pip

In [ ]:
# @title platform, pip

import platform

node = platform.node()
print(f"hostname: {node}")

if "dsw-" in node or "autodl" in node:
  %env HF_ENDPOINT https://hf-mirror.com

  if "autodl" in node:
      !source /etc/network_turbo

# @markdown pip install datasets transformers modelscope accelerate bitsandbytes peft trl -qq
!pip install rich datasets transformers accelerate bitsandbytes peft trl -qq
# modelscope

from rich.console import Console
console = Console()

# may need to restart kernel if dataset is not pre-installed
try:
  import datasets
except ValueError as exc:
  console.print(exc)
  console.print("\t[red bold]Try to restart kernel to fix this")

hostname: eaf5859022f2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which 

In [ ]:
!pip install -qq set-env-colab-kaggle-dotenv huggingface-hub wandb
from set_env import set_env
from huggingface_hub import login
import wandb

%env WANDB_PROJECT wandb-project
HF_TOKEN = set_env("HF_TOKEN")
WANDB_API_KEY = set_env("WANDB_API_KEY")
login(HF_TOKEN)
wandb.login(key=WANDB_API_KEY, anonymous="allow")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00
env: WANDB_PROJECT=wandb-project
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: Currently logged in as: yucongo-fmt. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 导入库

In [ ]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
# 将JSON文件转换为Dataset
# df = pd.read_json('huanhuan.json')

url = 'https://raw.githubusercontent.com/datawhalechina/self-llm/master/dataset/huanhuan.json'
df = pd.read_json(url)

ds = Dataset.from_pandas(df[:500])

In [ ]:
ds, ds[:3]

(Dataset({
     features: ['instruction', 'input', 'output'],
     num_rows: 500
 }),
 {'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
   '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
   '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。'],
  'input': ['', '', ''],
  'output': ['嘘——都说许愿说破是不灵的。', '你们俩话太多了，我该和温太医要一剂药，好好治治你们。', '出来走走，也是散心。']})

# 处理数据集

In [ ]:
# repo_id = 'baichuan-inc/Baichuan2-7B-Chat'
repo_id = 'WHJ1998/Baichuan2-7B-Chat'
#
tokenizer = AutoTokenizer.from_pretrained(repo_id, use_fast=False, trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

tokenizer

BaichuanTokenizer(name_or_path='WHJ1998/Baichuan2-7B-Chat', vocab_size=125696, model_max_length=4096, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [ ]:
from textwrap import dedent
template = dedent("""
  <|im_start|>system
  {system_message}<|im_end|>
  <|im_start|>user
  {prompt}<|im_end|>
  <|im_start|>assistant
  """).lstrip("\n")

system_message = '''现在你要扮演皇帝身边的女人--甄嬛。'''
template

'<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n'

In [ ]:
def process_func(example):
    MAX_LENGTH = 256    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["<|im_start|>system", "现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>" + "\n<|im_start|>user\n" + example["instruction"] + example["input"] + "<|im_end|>\n"]).strip()+"\n\nAssistant: ",add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(example["output"]+tokenizer.eos_token, add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]   # Qwen的特殊构造就是这样的
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
def proc_func(sample):
  MAX_LENGTH = 256
  text = template.format(system_message=system_message, prompt=sample["instruction"]) + sample["output"] + tokenizer.eos_token
  return {"text": text}

In [ ]:
print(proc_func(ds[1])['text'])

<|im_start|>system
现在你要扮演皇帝身边的女人--甄嬛。<|im_end|>
<|im_start|>user
这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。<|im_end|>
<|im_start|>assistant
你们俩话太多了，我该和温太医要一剂药，好好治治你们。</s>


In [ ]:
["<|im_start|>system", "现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>" + "\n<|im_start|>user\n" + '''===example["instruction"] + example["input"]==''' + "<|im_end|>\n"]

['<|im_start|>system',
 '现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>\n<|im_start|>user\n===example["instruction"] + example["input"]==<|im_end|>\n']

In [ ]:
'''example["output"]===+'''+tokenizer.eos_token

'example["output"]===+</s>'

In [ ]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [ ]:
dataset = ds.map(proc_func)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:

dataset, dataset[0], dataset[0]['text']

(Dataset({
     features: ['instruction', 'input', 'output', 'text'],
     num_rows: 500
 }),
 {'instruction': '小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  'input': '',
  'output': '嘘——都说许愿说破是不灵的。',
  'text': '<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛。<|im_end|>\n<|im_start|>user\n小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>\n<|im_start|>assistant\n嘘——都说许愿说破是不灵的。</s>'},
 '<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛。<|im_end|>\n<|im_start|>user\n小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>\n<|im_start|>assistant\n嘘——都说许愿说破是不灵的。</s>')

In [ ]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))

 <|im_start|>system
现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>
<|im_start|>user
小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>

Assistant: 嘘——都说许愿说破是不灵的。</s>


In [ ]:
tokenized_id[0]['input_ids']

[92655,
 92647,
 1418,
 92484,
 13387,
 92647,
 92574,
 24399,
 5,
 2016,
 8897,
 19289,
 11691,
 4131,
 10282,
 63,
 63,
 45860,
 72,
 92655,
 92647,
 1418,
 92484,
 1520,
 92647,
 92574,
 5,
 92655,
 92647,
 1418,
 92484,
 13387,
 92647,
 92574,
 8589,
 5,
 17875,
 65,
 9174,
 93245,
 92608,
 4029,
 92643,
 46931,
 65,
 17728,
 10701,
 17875,
 92575,
 92593,
 97186,
 41034,
 65,
 10205,
 2055,
 4948,
 92673,
 92673,
 29703,
 1935,
 92655,
 92647,
 1418,
 92484,
 1520,
 92647,
 92574,
 5,
 5,
 64286,
 92345,
 92311,
 95849,
 1935,
 12986,
 63065,
 92488,
 93197,
 5719,
 59319,
 66,
 2]

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

' 你们俩话太多了，我该和温太医要一剂药，好好治治你们。</s>'

# 创建模型

In [ ]:
import torch

!pip install xformers -qq

# repo_id = 'baichuan-inc/Baichuan2-7B-Chat'
# repo_id = 'WHJ1998/Baichuan2-7B-Chat'  # sharded? os.error: no generation_config.json

model = AutoModelForCausalLM.from_pretrained(
  # 'baichuan-inc/Baichuan2-7B-Chat',  # colab 12GB free tier RAM OOM, model file 15GB too big
  repo_id,
  trust_remote_code=True,
  torch_dtype=torch.half,
  device_map="auto",
)
model.generation_config = GenerationConfig.from_pretrained(repo_id)
model

Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

BaichuanForCausalLM(
  (model): BaichuanModel(
    (embed_tokens): Embedding(125696, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (self_attn): Attention(
          (W_pack): Linear(in_features=4096, out_features=12288, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): NormHead()
)

In [ ]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [ ]:
model.dtype

torch.float16

# lora

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["W_pack", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'o_proj', 'W_pack', 'gate_proj', 'down_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [ ]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='WHJ1998/Baichuan2-7B-Chat', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'o_proj', 'W_pack', 'gate_proj', 'down_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [ ]:
model.print_trainable_parameters()

trainable params: 17,891,328 || all params: 7,523,864,576 || trainable%: 0.23779439168895536


# 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="./output/Baichuan6",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    logging_steps=10,
    num_train_epochs=4,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

NotImplementedError: Cannot copy out of meta tensor; no data!

In [ ]:
trainer.train()

In [ ]:
model.eval()
inputs = tokenizer("<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n".format("你是谁", "").strip() + "\nassistant\n ", return_tensors="pt")
outputs = model.generate(**inputs.to(model.device), max_new_tokens=100, eos_token_id=2)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

peft_model_id = "output/Baichuan5/checkpoint-1100"  # 这里我训练出效果最好的一版是 checkpoint-600，所以调用了这个，大家可以根据自己情况选择
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained("baichuan-inc/Baichuan2-7B-Chat", use_fast=False, trust_remote_code=True)
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
model.eval()
input = tokenizer("<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n".format("你是谁", "").strip() + "\nassistant\n ", return_tensors="pt").to(model.device)

max_length = 512

outputs = model.generate(
    **input,
    max_length=max_length,
    eos_token_id=2,
    do_sample=True,
    repetition_penalty=1.3,
    no_repeat_ngram_size=5,
    temperature=0.1,
    top_k=40,
    top_p=0.8,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
model.eval()
input = tokenizer("<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n".format("这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。", "").strip() + "\nassistant\n ", return_tensors="pt").to(model.device)

max_length = 512

outputs = model.generate(
    **input,
    max_length=max_length,
    eos_token_id=2,
    do_sample=True,
    repetition_penalty=1.3,
    no_repeat_ngram_size=5,
    temperature=0.1,
    top_k=40,
    top_p=0.8,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))